In [0]:
%fs ls adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/

path,name,size,modificationTime
adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/countries.json,countries.json,114118,1708866360843
adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/customers.json,customers.json,69292,1708866362702
adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/orders.json,orders.json,671174,1708866374748
adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/products.json,products.json,994,1708866361061
adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/sales.json,sales.json,4154197,1708866423416


In [0]:
%python
configs = {
  "fs.adl.oauth2.access.token.provider.type": "CustomAccessTokenProvider",
  "fs.adl.oauth2.access.token.custom.provider": spark.conf.get("spark.databricks.passthrough.adls.tokenProviderClassName")
}
dbutils.fs.mount(
source = "adl://cadlstore2hxco2cjxvxkw.azuredatalakestore.net/",
mount_point = "/mnt/datalake",
extra_configs = configs)

True

In [0]:
%sql
CREATE TABLE countries
USING json
OPTIONS (
path "/mnt/datalake/countries.json"
);
CREATE TABLE customers
USING json
OPTIONS (
  path "/mnt/datalake/customers.json"
);
CREATE TABLE orders
USING json
OPTIONS (
  path "/mnt/datalake/orders.json"
);
CREATE TABLE products
USING json
OPTIONS (
  path "/mnt/datalake/products.json"
);
CREATE TABLE sales
USING json
OPTIONS (
  path "/mnt/datalake/sales.json"
)

Number of countries that use the currency code 'EUR' to conduct business according to countries data?


In [0]:
%sql
SELECT count(currency) FROM countries
WHERE currency = 'EUR';

count(currency)
28


Number of customers they deal with in the country of Indonesia ?

In [0]:
%sql
SELECT count(DISTINCT customerId) FROM (
  SELECT * FROM customers c 
  JOIN countries ct 
  ON c.country = ct.country
  WHERE ct.name = "Indonesia"
)

count(DISTINCT customerId)
1


Currency code that had the maximum number sales transaction records for sales orders created in 2019? 

In [0]:
%sql
SELECT ct.currency, count(s.saleId) AS nb_sales FROM sales s
  JOIN orders o
  ON s.orderId = o.orderId
  JOIN customers c 
  ON c.customerId = o.customerId
  JOIN countries ct 
  ON c.country = ct.country
WHERE EXTRACT(YEAR FROM CAST(o.date AS DATE)) = 2019
GROUP BY ct.Currency
ORDER BY nb_sales DESC;

currency,nb_sales
EUR,3399
XCD,1609
USD,1295
XOF,1256
XAF,1133
GBP,663
AUD,470
CNY,459
PGK,457
SHP,440


Customers which have one of the letters q, x, or z in there name.

In [0]:
%sql
SELECT count(customerId) FROM customers
WHERE active = 'true'
AND LOWER(name) REGEXP '[qxz]'

count(customerId)
39


Top customer in terms of confirmed order quantities (the sum of sales quantities in the order) for products manufactured in the OCEANIA region.

In [0]:
%sql
select c.customerId, c.name, ct.name, qy.orderId, qy.total_quantity from customers c 
join countries ct on c.country = ct.country
join     
(select o.orderId, o.customerId, sum(s.quantity) as total_quantity
from sales s
  join orders o on s.orderId = o.orderId
  join products p on p.productId = s.productId
  join countries ct on ct.Country = p.ManufacturedCountry
where ct.region = 'OCEANIA'
group by o.orderId, o.customerId ) qy on qy.customerId = c.customerId
order by qy.total_quantity desc;

customerId,name,name,orderId,total_quantity
243,Quentin Harvey,"Congo, Repub. of the",6905,29
300,Joseph Trujillo,Cameroon,3587,24
351,Burton Key,Papua New Guinea,2131,22
370,Keane Rich,Kyrgyzstan,6550,21
116,Chandler Perkins,Georgia,11203,20
287,Kaseem Gibbs,Guatemala,1223,20
117,August Ramirez,Saint Vincent and the Grenadines,11096,20
368,Fuller Ball,Netherlands,1592,19
386,Ferdinand Campbell,Kazakhstan,9677,19
45,Peter Hubbard,Bolivia,8189,19


The customer name which has the maximum cumulative Quantity from the Sales records.

In [0]:
%sql
select c.name, sum(s.quantity) as total_quantity from customers c
join orders o 
on c.customerId = o.customerId
join sales s 
on o.orderId = s.orderId 
where c.active = 'true'
group by c.name
order by total_quantity desc;

name,total_quantity
Nicholas Craig,1056
Jameson Glover,970
Gabriel Cabrera,930
Allistair Warner,929
Demetrius Heath,928
Peter Diaz,910
Timothy Medina,890
Cadman Whitley,889
Joshua Kinney,878
Herrod Brennan,834


Aggregated weight in grams of the sales where the sales transactions happened in the Currency CAD.

In [0]:
%sql
select sum(p.WeightGrams) from sales s 
  join products p 
  on s.ProductId = p.ProductId
  join orders o 
  on o.OrderId = s.OrderId
  join customers c 
  on c.CustomerId = o.CustomerId
  join countries ct 
  on c.Country = ct.Country
where ct.Currency = 'CAD'

sum(WeightGrams)
49096


How many customer orders were placed in 2018, for customers in regions EASTERN EUROPE and WESTERN EUROPE?

In [0]:
%sql
select count(*) from orders o
  join customers c 
  on o.CustomerId = c.CustomerId
  join countries ct 
  on c.Country = ct.Country
where (ct.Region = 'EASTERN EUROPE' or ct.Region = 'WESTERN EUROPE')
and extract(year from cast(o.date as DATE)) = '2018'

count(1)
1014


The total cumulative sales quantity for the customers in the countries Thailand, Indonesia, and Vietnam?

In [0]:
%sql
select sum(s.Quantity) from sales s 
  join orders o 
  on o.OrderId = s.OrderId
  join customers c 
  on c.CustomerId = o.CustomerId
  join countries ct 
  on c.Country = ct.Country
where ct.name in ('Thailand', 'Indonesia', 'Vietnam')

sum(Quantity)
1195


Which order had the maximum cumulative product quantities (sum of product quantities in each sale included in the order) on '2018-06-05'?

In [0]:
%sql
select o.OrderId, sum(s.Quantity) as total_quantity from orders o 
  join sales s 
  on o.OrderId = s.OrderId
where o.Date = '2018-06-05'
group by o.OrderId
order by total_quantity desc;

OrderId,total_quantity
2253,46
2248,43
2242,41
2240,33
2232,32
2249,29
2236,28
2235,28
2252,28
2247,26
